In [ ]:
import os,sys
import cv2
import numpy as np
from numpy import shape
import pandas as pd
import open_clip
from tqdm import tqdm
import torch
import open_clip
import logging
import traceback
from blip.models.blip import blip_decoder, BLIP_Decoder
from PIL import Image,ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
Image.MAX_IMAGE_PIXELS = None
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('tagsets')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, ne_chunk
import string
from concurrent.futures import ThreadPoolExecutor, as_completed
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
import inspect
import torch.nn.functional as F
import codecs
import unicodedata
from sentence_transformers import SentenceTransformer
import albumentations as A
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from unidecode import unidecode
import re
import enchant
d = enchant.Dict("en_US")

def rm_punct(s)->str:
    return s.translate(str.maketrans('', '', string.punctuation))
def rm_white(s)->str:
    return  " ".join(s.split())
def rm_sw(s)->str:
    sw = set(stopwords.words("english"))
    wt = word_tokenize(s)
    t = ' '.join(w for w in wt if w not in sw)
    return t
def rm_spec(s)->str:
    return re.sub('\s+', ' ', (re.sub('_', '', (re.sub('[^a-zA-z0-9\s]', '', unidecode(s)))))).strip().lower()
def w_ec(s):
    return d.check(str(s))

class CFG:
    debug = False
    image_path = "P:/sd_prompts/images/"
    caption_path = image_path
    tags_save_path = image_path
    tag_list_path = "P:/sd_prompts/text_tags/"
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model_name = 'ViT-H-14/laion2b_s32b_b79k'
    image_embedding = 2048
    text_encoder_model = "distilbert-base-uncased"
    text_embedding = 2048
    text_tokenizer = "distilbert-base-uncased"
    sentence_model_path = "E:/j_diff/ci/sentence-transformers/all-MiniLM-L6-v2"
    embedding_length = 384

    blip_model: BLIP_Decoder = None
    blip_image_eval_size: int = 384
    blip_accuracy_confidence: float = 0.7
    blip_max_cap_len: int = 120
    blip_min_cap_len: int = 20
    blip_rep_pen: float = 1.2
    blip_model_type: str = 'large'
    blip_model_url: str = 'https://storage.googleapis.com/sfr-vision-language-research/BLIP/models/model_large_caption.pth'
    blip_num_beams: int = 1
    blip_offload: bool = False
    clip_model_name: str = 'ViT-H-14/laion2b_s32b_b79k'
    clip_model_path: str = None
    clip_model = None
    clip_preprocess = None
    clip_sensitivity: int = 30
    pretrained = True # for both image encoder and text encoder
    block_size = 2048
    clip_model_list = \
                ['ViT-B-32/openai', #                  #       0
                'ViT-B-32/laion400m_e31',#             #       1
                'ViT-B-32/laion400m_e32',#             #       2
                'ViT-B-32/laion2b_e16',#               #       3
                'ViT-B-32/laion2b_s34b_b79k',#         #       4
                'ViT-B-32-quickgelu/openai',#          #       5
                'ViT-B-32-quickgelu/laion400m_e31',#   #       6
                'ViT-B-32-quickgelu/laion400m_e32',#   #       7
                'ViT-B-16/openai',#                    #       8
                'ViT-B-16/laion400m_e31',#             #       9
                'ViT-B-16/laion400m_e32',#             #       10
                'ViT-B-16-plus-240/laion400m_e31',#    #       11
                'ViT-B-16-plus-240/laion400m_e32',#    #       12
                'ViT-L-14/openai',#                    #       13
                'ViT-L-14/laion400m_e31',#             #       14
                'ViT-L-14/laion400m_e32',#             #       15
                'ViT-L-14/laion2b_s32b_b82k',#     `   #       16
                'ViT-L-14-336/openai',#                #       17
                'ViT-H-14/laion2b_s32b_b79k',#         #       18
                'ViT-g-14/laion2b_s12b_b42k',#         #       19
                'clip-vit-large-patch14-336/openai']#  #       20

st_model = SentenceTransformer(CFG.sentence_model_path)

In [ ]:
if CFG.blip_model is None:
    print("BLIP")
    blip_path = os.path.dirname(inspect.getfile(blip_decoder))
    configs_path = os.path.join(os.path.dirname(blip_path), 'configs')
    med_config = os.path.join(configs_path, 'med_config.json')
    blip_model = blip_decoder(
        pretrained=CFG.blip_model_url,
        vit='large',
        med_config=med_config
    )
    blip_model.eval()
    blip_model = blip_model.to(CFG.device)
    blip_model = blip_model
else:
    blip_model = CFG.blip_model

In [ ]:
if CFG.clip_model is None:
    clip_model_name, clip_model_pretrained_name = CFG.clip_model_list[16].split('/', 2)
    clip_model, _, clip_preprocess = open_clip.create_model_and_transforms(
        clip_model_name, 
        pretrained = clip_model_pretrained_name, 
        precision='fp16' if CFG.device == 'cuda' else 'fp32',
        device = CFG.device,
        jit=False,
        cache_dir = CFG.clip_model_path
    )
    clip_model.to(CFG.device).eval()
    clip_tokenize = open_clip.get_tokenizer(clip_model_name)

In [ ]:
core_image_datum = []
tag_list         = []
cat_list           = []
tag_path_files = [  f for f 
                    in os.listdir(CFG.tag_list_path[:-1:]) 
                    if os.path.isfile(CFG.tag_list_path+f)]
tag_text_files = [  CFG.tag_list_path+f for f 
                    in tag_path_files 
                    if f[-(f[::-1].find('.')):] 
                    in ['txt','cap']]

In [ ]:
def block_proc(tags):
    blocks = np.array_split(tags, max(1, (len(tags)/CFG.block_size)))
    block_embeds = []
    for block in tqdm(blocks):
        text_tokens = clip_tokenize(block).to(CFG.device)
        with torch.no_grad(), torch.cuda.amp.autocast():
            text_features = clip_model.encode_text(text_tokens)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            text_features = text_features.detach().half().cpu().numpy()
        for i in range(text_features.shape[0]):
            block_embeds.append(text_features[i])
        cat_embed = torch.stack([torch.from_numpy(t) for t in block_embeds]).to(CFG.device)
        cat_list.append(cat_embed)
    return cat_list,tags

def tag_blocks():
    for tag_file in tqdm(tag_text_files,desc="Tag Files"):
        tags = []
        tags.clear()
        with open(tag_file,"rt",encoding='utf-8') as me:
            txt_data = bytes(me.read(),'utf-8')
            txt_data = unidecode(codecs.decode(txt_data))
            str_sanity = str(txt_data).split('\n')
            tags = [    str(rm_punct(rm_spec(rm_white(x)))).strip() for x 
                            in str_sanity]
        block_proc(tags)
        for x in tags:
            tag_list.append(x)
    return cat_list,tag_list

cat_list,tag_list = tag_blocks()

In [ ]:
def get_cap(pil_image,imagefile)->list:
    try:    
        captions = ""
        sens_list = [0.05,0.1,0.15]
        caption_list = []
        size = CFG.blip_image_eval_size
        np_pil_image = np.array(pil_image)
        b_img = transforms.Compose([
            transforms.Resize((size,size),interpolation=InterpolationMode.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
            ])
        with torch.no_grad():
            ti = b_img(pil_image).unsqueeze(0).to(CFG.device)
            for b_sen in tqdm(sens_list):
                for b_len in range(10,30,5):
                    captions =blip_model.generate(
                                ti, 
                                sample = False, 
                                top_p = b_sen,
                                max_length = CFG.blip_max_cap_len, 
                                min_length = b_len, 
                                repetition_penalty = 1.4,
                                num_beams = CFG.blip_num_beams
                                )                    
                    captions = str(captions[0])
                    captions = rm_sw(captions)
                    captions = rm_punct(captions)
                    captions = rm_white(captions)
                    captions = str(captions) +'\n'
                    print(captions)
                    caption_list.append(captions)
        f_name = os.path.basename(imagefile)
        with open(CFG.tags_save_path+f_name[:(len(f_name))-1-len(f_name[-(f_name[::-1].find('.')):])]+"_captions"+".txt",'wt') as fi:
            fi.writelines(caption_list)
    except Exception as e:
            logging.error(traceback.format_exc())            
    finally:
        pass    
    return caption_list

In [ ]:
def blip_clip_cap_tag(imagefile,cat_list):
    try:
        img = cv2.imread(imagefile)
        img2 = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(img2)
        match_list = []
        match_list.clear()
        val_list = []
        val_list.clear()
        uniq_match = []
        uniq_match.clear()
        matches = []
        matches.clear()        
        caption = get_cap(pil_image,imagefile)
        for tags in cat_list:
            with torch.no_grad(), torch.cuda.amp.autocast():
                    image = clip_preprocess(pil_image).unsqueeze(0).to(CFG.device)
                    image_features_n = clip_model.encode_image(image)
                    image_features_n =  F.normalize(image_features_n,dim=-1)
                    text_embeddings_n = F.normalize(kitty, dim=-1)
                    dot_similarity =  image_features_n @ text_embeddings_n.T
                    values, indices = dot_similarity.float().cpu().topk(k=1,largest=True, sorted=True)
                    indices.detach().float().cpu().numpy()
                    values.detach().float().cpu()
                    val_list = [abs(int(torch.IntTensor.item(x)*100)) for x in values[0,:][::]]
                    matches = [tags[x] for x in indices[0,:][::]]
                    for i in range(len(val_list)):
                        if val_list[i]>=CFG.clip_sensitivity:
                            match_list.append(str(val_list[i])+'_'+str(matches[i]))
                    values.detach().float().cpu().numpy()
                    del indices
                    del values
                    del val_list
                    del matches
        print(len(match_list))
        print(match_list)
        # f_name = os.path.basename(imagefile)
        # if os.path.isfile(CFG.tags_save_path+f_name[:(len(f_name))-1-len(f_name[-(f_name[::-1].find('.')):])]+".txt"):
        #     with open(CFG.tags_save_path+f_name[:(len(f_name))-1-len(f_name[-(f_name[::-1].find('.')):])]+".txt",'a') as fi:
        #         for x in match_list:
        #             fi.write(str(x)+'\n')
        # elif not os.path.isfile(CFG.tags_save_path+f_name[:(len(f_name))-1-len(f_name[-(f_name[::-1].find('.')):])]+".txt"):
        #     with open(CFG.tags_save_path+f_name[:(len(f_name))-1-len(f_name[-(f_name[::-1].find('.')):])]+".txt",'wt') as fi:
        #         for x in match_list:
        #             fi.write(str(x)+'\n')
    except Exception as e:
        logging.error(traceback.format_exc())            
    finally:
        pass

In [ ]:
# def img_tag_proc(img_list):
#     with ThreadPoolExecutor(8) as executor:
#         futures = [executor.submit(blip_clip_cap_tag, i) for i in img_list]
#         for _ in as_completed(futures):
#             status_bar.update(n=1)


# imgs_len = len(image_list)
# status_bar = tqdm(total=imgs_len, desc='Image Processing')
for imagefile in tqdm(image_list):
    blip_clip_cap_tag(imagefile,cat_list)